In [25]:
# Import necessary libraries
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import networkx as nx
import os,sys


notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))
from engine.core_rec import *


In [26]:

# Import custom modules
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data

# Workaround for __file__ not being defined in Jupyter notebooks
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))


# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert data to torch tensors
x = data.x
edge_index = data.edge_index

# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)

# Add train_mask and test_mask
num_nodes = x.size(0)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = True
test_mask = ~train_mask

data.train_mask = train_mask
data.test_mask = test_mask

# Add labels (y) if not already present
num_classes = dataset.num_classes
data.y = data.y if data.y is not None else torch.randint(0, num_classes, (num_nodes,), dtype=torch.long)


In [27]:
adj_matrix = nx.to_numpy_array(nx.from_edgelist(edge_index.t().tolist()))
train_dataset = GraphDataset(adj_matrix[train_mask])
test_dataset = GraphDataset(adj_matrix[test_mask])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [42]:

# Initialize the model
num_layers = 1
d_model = 128
num_heads = 8
d_feedforward = 512
input_dim = 2708 # Ensure this matches the number of features in the dataset
model = GraphTransformerV2(num_layers, d_model, num_heads, d_feedforward, input_dim)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=100)
# Evaluate the model
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        # print(f"Input shape: {inputs.shape}")  # Add this line to check the input shape
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        # print(f"Preds shape: {preds.shape}, Targets shape: {targets.shape}")  # Debugging shapes
        targets = torch.argmax(targets, dim=1)  # Convert one-hot encoded targets to class indices
        all_preds.extend(preds.cpu().numpy().flatten())  # Ensure preds are flattened
        all_labels.extend(targets.cpu().numpy().flatten())  # Ensure targets are flattened
        # print(f"Current length of all_preds: {len(all_preds)}, all_labels: {len(all_labels)}")  # Debugging lengths

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Debugging: Print final lengths of all_preds and all_labels
# print(f"Final length of all_preds: {len(all_preds)}")
# print(f"Final length of all_labels: {len(all_labels)}")

# Ensure both arrays are 1-dimensional
all_preds = all_preds.flatten()
all_labels = all_labels.flatten()

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

TypeError: GraphTransformerV2.__init__() missing 1 required positional argument: 'output_dim'